# make_blobs
`data, label = make_blobs(n_features=2, n_samples=100, centers=3, random_state=3, cluster_std=[0.8, 2, 5])` 函数主要是为了生成数据集

- `n_features`表示每一个样本有多少特征值
- `n_samples`表示样本的个数
- `centers`是聚类中心点的个数，可以理解为label的种类数
- `random_state`是随机种子，可以固定生成的数据
- `cluster_std`设置每个类别的方差

In [5]:
from sklearn.datasets import make_blobs
data, label = make_blobs(n_features=2, n_samples=15, centers=2, random_state=20210714, cluster_std=[0.6,0.7] )
print(data)
print(label)

[[-5.86531981  8.95266886]
 [-6.48309273  7.93586256]
 [-0.03865178 -2.15324279]
 [-0.92537911 -1.31508482]
 [-1.22163176 -2.04121749]
 [-6.83497904  8.09078853]
 [-6.3743914   7.23415993]
 [-6.20132316  8.79222996]
 [-0.39311256 -1.10224094]
 [-6.86972482  7.0158355 ]
 [ 0.23019253 -1.4388951 ]
 [-6.51989119  8.84318314]
 [ 0.40411778 -0.75002896]
 [-6.46479379  7.85399464]
 [-0.45894291 -0.71611558]]
[0 0 1 1 1 0 0 0 1 0 1 0 1 0 1]


# predict_proba、predict、decision_function

三种方法都是用来展示模型对于输入样本评判结果的

在sklearn中，对于训练好的分类模型，模型都有一个classes_属性，classes_属性中按顺序保存着训练样本的类别标记

[原文](https://blog.csdn.net/cxx654/article/details/106727812)

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression

x = np.array([[-1, -1],[-2, -1],[1, 1],[2, 1],[-1, 1],[-1, 2],[1, -1],[1, -2]])
y = np.array([2, 2, 3, 3, 0, 0, 1, 1])
 
clf = LogisticRegression()
clf.fit(x, y)
clf.classes_

array([0, 1, 2, 3])

In [2]:
x = np.array([[-1, -1],[-2, -1],[1, 1],[2, 1],[-1, 1],[-1, 2],[1, -1],[1, -2]])
y = np.array([6, 6, 2, 2, 4, 4, 8, 8])
 
clf = LogisticRegression()
clf.fit(x, y)
clf.classes_

array([2, 4, 6, 8])

- `predict_proba`模型预测输入样本属于每种类别的概率，概率和为1，每个位置的概率分别对应classes_中对应位置的类别标签。以上述类别标签为[2 4 6 8]的那个分类器为例，查看一下分类模型预测的概率。

In [4]:
clf.predict_proba([[-1, -1]])

array([[0.03997361, 0.12532009, 0.63284608, 0.20186022]])

显然第三个概率最大，也对应着classes_的类别6，predict_proba输出概率最大值索引位置对应的classes_元素就是样本所属的类别

- `predict`

In [5]:
clf.predict([[-1, -1]])

array([6])

- `decision_function`帮助文档中给出的解释是“The confidence score for a sample is the signed distance of that sample to the hyperplane.”。意思就是使用样本到分隔超平面的有符号距离来度量预测结果的置信度

1.他是谁
看一下支持向量机SVM中关于decision_function的解释是怎样的？

说了两件事情，其一是说评估样本X的的decision_function（等于没说，哈哈哈），其二是说，如果decision_dunction_shape='ovr',则输出的decison_function形状是(n_samples, n_classes), n_samples是输入样本的数量，n_classes是训练样本的类别数。这里再补充一点，如果decision_dunction_shape='ovo,则输出的decison_function形状是(n_samples, n_classes * (n_classes - 1) / 2)。‘ovr’和‘ovo’又是啥？莫急，莫急。暂且知道是用于训练多分类的就行。

2.他从哪里来？
SVM中通过支持向量来选择分隔超平面，分隔超平面将训练样本分为正反两派

3.他到哪里去？

说了半天，decison_function这玩意到底有啥用？莫急，莫急。下面先说一下上面提到的'ovr'和'ovo'分别是什么东东？

我们常见的分类器，比如LR和SVM都是只能支持二分类的，回想一下LR分类器，通过判断线性模型的预测结果是否大于0，进而判断sigmoid的输出结果是否大于0.5来判断模型属于正类还是负类。SVM也一样，前面讲了，SVM通过分隔超平面将样本分到两边去，也就是进行二分类。那么怎么能将二分类的分类算法应用到多分类任务上去呢？这就是‘ovr’和‘ovo’要解决的问题。

`'ovr'`：全称是One-vs-Rest。就是一个人和对面一群人干一次架（群殴）。假如我们训练数据中包含[0, 1, 2, 3]四个分类，那么分别将0, 1, 2, 3作为正样本，其余的123， 023， 013， 012作为负样本，训练4个分类器，每个分类器预测的结果表示属于对应正类也就是0， 1， 2， 3 的概率。这样对于一个输入样本就相当于要进行4个二分类，然后取输出结果最大的数值对应的classes_类别。

`'ovo'`：全称是One-vs-One。就是一个人分别和对面的每个人干一次架（单挑，车轮战术）。同样，假如我们训练数据中包含[0, 1, 2, 3]四个分类，先将类别0作为正样本，类别1，类别2，类别3依次作为负样本训练3个分类器，然后以类别1为正样本，类别0，类别2， 类别3作为负样本训练3个分类器，以此类推。由于类别0为正样本，类别1为负样本和类别1为正样本、类别0为负样本实质上是一样的，所以不需要重复训练。

通过上面的描述可知，假如训练样本有n_classes个类别，则'ovr'模式需要训练n_classes个分类器，‘ovo’模式需要训练n_classes * (n_classes - 1) / 2 个分类器。那么问题来了，有多少个分类器是不是就得有多少个分隔超平面，有多少个分隔超平面是不是就得有多少个decision_function值。这也就对应了“他是谁？”那部分所说的decison_function输出形状的描述。

- 1、二分类的decison_function

二分类模型中，decision_function返回的数组形状等于样本个数，也就是一个样本返回一个decision_function值。并且，此时的decision_function_shape参数失效 ，因为只需要训练一个分类器就行了，就不存在是单挑还是群殴的问题了。下面以SVM二分类的实例来看一下结果：

In [7]:
from sklearn.svm import SVC
x = np.array([[1,2,3],
              [1,3,4],
              [2,1,2],
              [4,5,6],
              [3,5,3],
              [1,7,2]])
 
y = np.array([3, 3, 3, 2, 2, 2])
 
clf = SVC(probability=True)
clf.fit(x, y)
print(clf.decision_function(x))
print(clf.predict(x))
# 返回array([2, 3])，其中2为negetive，3为positive
print(clf.classes_)

[ 1.00089036  0.64493601  0.97960658 -1.00023781 -0.9995244  -1.00023779]
[3 3 3 2 2 2]
[2 3]


在二分类的情况下，分类模型的decision_function返回结果的形状与样本数量相同，且返回结果的数值表示模型预测样本属于positive正样本的可信度。并且，二分类情况下classes_中的第一个标签代表是负样本，第二个标签代表正样本。

还记得前面讲过的decision_function是有符号的吧，大于0表示正样本的可信度大于负样本，否则可信度小于负样本。所以对于前3个样本，decison_function都认为是正样本的可信度高，后3个样本是负样本的可信度高。那么再看一下predict的结果，前3个预测为正样本3（ps：二分类情况下正样本对应的是classes_中的第二个类别），后3个样本预测为负样本2。再看一下predict_proba预测的样本所属的类别概率，可以看到前3个样本属于类别3的概率更大，后3个样本属于类别2的概率更大。

2、多分类的decision_function

多分类模型中，decision_function返回的数组形状依据使用的模式是‘ovr’还是‘ovo’而分别返回n_classes个和n_classes * (n_classes - 1) / 2个数值。下面以SVM多分类的实例来看一下结果：

`One-vs-Rest`多分类实例：

In [8]:
X = np.array(
    [
        [-1, -1],
        [-2, -1],
        [1, 1],
        [2, 1],
        [-1, 1],
        [-1, 2],
        [1, -1],
        [1, -2]
    ]
)
y = np.array([2, 2, 3, 3, 0, 0, 1, 1])
# SVC多分类模型默认采用ovr模式
clf = SVC(probability=True, decision_function_shape="ovr")
clf.fit(X, y)
 
# 计算样本距离每个分类边界的距离
# One-vs-Rest 选择decision_function的得分[0-Rest, 1-Rest, 2-Rest, 3-Rest]最大的作为分类结果
print("decision_function:\n", clf.decision_function(X))
# precidt预测样本对应的标签类别
print("predict:\n", clf.predict(X))
# predict_proba 预测样本对应各个类别的概率
print("predict_proba:\n", clf.predict_proba(X)) #这个是得分,每个分类器的得分，取最大得分对应的类。
print("classes_:", clf.classes_)

decision_function:
 [[ 0.85207453  1.88733077  3.2464045  -0.2013789 ]
 [ 1.8755707   0.82458773  3.24999265 -0.18797977]
 [ 1.8873491   0.8520616  -0.20137224  3.24640161]
 [ 0.82459296  1.87556154 -0.18798785  3.2499953 ]
 [ 3.24640646 -0.20136156  1.88723018  0.85210243]
 [ 3.2500016  -0.18796346  0.82451365  1.87561044]
 [-0.20135491  3.24640358  0.85211537  1.88721187]
 [-0.18797154  3.25000425  1.8756196   0.82450842]]
predict:
 [2 2 3 3 0 0 1 1]
predict_proba:
 [[0.0228462  0.03534781 0.88634889 0.0554571 ]
 [0.02256957 0.02338434 0.88953071 0.06451538]
 [0.0349277  0.02294708 0.05551724 0.88660798]
 [0.02293709 0.02264523 0.06453674 0.88988095]
 [0.88657068 0.05535168 0.03523928 0.02283836]
 [0.88977798 0.06435016 0.02329769 0.02257417]
 [0.05548965 0.88634678 0.0228759  0.03528766]
 [0.06453234 0.88950293 0.02260892 0.02335581]]
classes_: [0 1 2 3]


`One-vs-One`多分类实例：

In [9]:
X = np.array(
    [
        [-1, -1],
        [-2, -1],
        [1, 1],
        [2, 1],
        [-1, 1],
        [-1, 2],
        [1, -1],
        [1, -2]
    ]
)
y = np.array([2, 2, 3, 3, 0, 0, 1, 1])
# SVC多分类模型默认采用ovr模式
clf = SVC(probability=True, decision_function_shape="ovo")
clf.fit(X, y)
 
# 计算样本距离每个分类边界的距离
# One-vs-One 按照decision_function的得分[01, 02, 03, 12, 13, 23]判断每个分类器的分类结果，然后进行投票
print("decision_function:\n", clf.decision_function(X))
# precidt预测样本对应的标签类别
print("predict:\n", clf.predict(X))
# predict_proba 预测样本对应各个类别的概率
print("predict_proba:\n", clf.predict_proba(X)) #这个是得分,每个分类器的得分，取最大得分对应的类。
print("classes_:", clf.classes_)

decision_function:
 [[-0.07609727 -1.00023294  0.27849207 -0.83425862  0.24756982  1.00006256]
 [ 0.12009339 -1.00023295  0.28451059 -0.99960621  0.00894039  0.99980785]
 [ 0.07622272  0.24756982 -0.83425862  0.27849207 -1.00023294 -0.99993521]
 [-0.12002341  0.00894039 -0.99960621  0.28451059 -1.00023295 -0.99993521]
 [ 1.00006256  0.83477528  0.99980309 -0.24727203 -0.27845811  0.07622272]
 [ 0.99980785  1.00046589  0.9998031  -0.00873952 -0.28445414 -0.12002341]
 [-0.99993521 -0.27845811 -0.24727203  0.99980309  0.83477528 -0.07609727]
 [-0.99993521 -0.28445414 -0.00873952  0.9998031   1.00046589  0.12009339]]
predict:
 [2 2 3 3 0 0 1 1]
predict_proba:
 [[0.0228462  0.03534781 0.88634889 0.0554571 ]
 [0.02256957 0.02338434 0.88953071 0.06451538]
 [0.0349277  0.02294708 0.05551724 0.88660798]
 [0.02293709 0.02264523 0.06453674 0.88988095]
 [0.88657068 0.05535168 0.03523928 0.02283836]
 [0.88977798 0.06435016 0.02329769 0.02257417]
 [0.05548965 0.88634678 0.0228759  0.03528766]
 [0.06

ovo模式下，4个类别的训练数据，需要训练6个二分类器，得到6个decition_function值，依照classes_的类别顺序，6个二分类器分别是[01, 02, 03, 12, 13, 23]，前面的数字表示正类，后面的表示负类。以decision_function的第一行输出结果为例：

```python
-0.07609727 对应 01分类器，且数值小于0，则分类结果为后者，即类别1
-1.00023294  对应 02分类器，且数值小于0，则分类结果为后者，即类别2
0.27849207  对应 03分类器，且数值大于0，则分类结果为前者，即类别0
-0.834258626  对应 12分类器，且数值小于0，则分类结果为后者，即类别2
0.24756982  对应 13分类器，且数值大于0，则分类结果为前者，即类别1
1.00006256 对应 23分类器，且数值大于0，则分类结果为前者，即类别2
 
最终得票数：{类别0: 1， 类别1: 2, 类别2: 3， 类别3: 0}
对以上分类结果voting投票，多数获胜，即最终分类结果为类别2。
```